### Imports library

In [1]:
from sklearn.metrics.pairwise import cosine_similarity
from gensim import corpora, models
import collections
import numpy as np
import smart_open
import pymorphy2
import random
import gensim
import json
import os

C:\Program Files\Anaconda3\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


### Corpus preparation

In [2]:
def get_percentile_counts(tokenized_lengths):
    minCount=np.min(tokenized_lengths)
    maxCount=np.max(tokenized_lengths)
    print(minCount)
    print(maxCount)
    for i in range(0,11):
        lower_border = i*(maxCount-minCount)/10+minCount
        upper_border = (i+1)*(maxCount-minCount)/10+minCount
        print(lower_border)
        print("Count: ",np.count_nonzero(tokenized_lengths*((tokenized_lengths>=lower_border)&(tokenized_lengths<=upper_border))))

def get_russian_lemma(token, lemmatizer):
    lemma = lemmatizer.parse(token.lower())[0]
    return lemma.normal_form

def get_lemmatized_sequence(sequence, lemmatizer):
    lemmas = []
    for token in sequence:
        lemma = get_russian_lemma(token, lemmatizer)
        if lemma.strip() != "":
            lemmas.append(lemma)
    return ' '.join(lemmas)

def get_lemmatized_document(fullfilename, lemmatizer):
    with open(fullfilename, 'r') as file:
        return get_lemmatized_sequence(gensim.utils.simple_preprocess(file.read().replace('\n', ' ')), lemmatizer)  

def build_lemmatized_corpora(path, targetfilename, lemmatizer):
    with open(targetfilename+'.cor', 'w') as corporafile, open(targetfilename+'.cfn', 'w') as filenamefile:
        for filename in os.listdir(path):
            line = get_lemmatized_document(path+os.sep+filename, lemmatizer)
            print(line, file=corporafile)
            print(filename,file=filenamefile)
            files_processed+=1
    print("Total files processed: ", files_processed)

def read_corpus(fname, tokens_only=False, preserve=False):
    with smart_open.smart_open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
            if preserve:
                yield line.decode()[:-1]
            elif tokens_only:
                yield gensim.utils.simple_preprocess(line)
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [i])


def split_corpus(full_corpus_filenames, full_corpus, split_fraction):
    random_filenames = np.random.rand(len(full_corpus_filenames)) < 1-split_fraction
    train_corpus=[]
    train_corpus_filenames=[]
    test_corpus=[]
    test_corpus_filenames = []

    for i in range(0, len(random_filenames)): 
        if i%4==0:#full_corpus_filenames[i][-5:-4]==str(1):
            test_corpus.append(full_corpus[i])
            test_corpus_filenames.append(full_corpus_filenames[i])
        else:
            train_corpus.append(full_corpus[i])
            train_corpus_filenames.append(full_corpus_filenames[i])
    return train_corpus_filenames, train_corpus, test_corpus_filenames, test_corpus

### Training models

In [3]:
def TrainLDAModel(train_corpus):
    # turn our tokenized documents into a id <-> term dictionary
    dictionary = corpora.Dictionary(train_corpus)
    
    # convert tokenized documents into a document-term matrix
    corpus = [dictionary.doc2bow(text) for text in train_corpus]

    # generate LDA model
    ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=lda_topics, id2word = dictionary, passes=20)
    return ldamodel, dictionary

def TrainLSIModel(train_corpus):
    # turn our tokenized documents into a id <-> term dictionary
    dictionary = corpora.Dictionary(train_corpus)
    
    # convert tokenized documents into a document-term matrix
    corpus = [dictionary.doc2bow(text) for text in train_corpus]

    # generate LSI model
    lsimodel = gensim.models.lsimodel.LsiModel(corpus, num_topics=lsi_topics, id2word = dictionary)
    return lsimodel, dictionary

def TrainTFIDFModel(train_corpus):
    # turn our tokenized documents into a id <-> term dictionary
    dictionary = corpora.Dictionary(train_corpus)
    
    # convert tokenized documents into a document-term matrix
    corpus = [dictionary.doc2bow(text) for text in train_corpus]

    # generate TFIDF model
    tfidfmodel = gensim.models.TfidfModel(corpus)
    return tfidfmodel, dictionary

def TrainW2VModel(train_corpus, dimensionality, iterations):
    sentences = train_corpus
    #sentences = SentenceModel(train_corpus)
    #model = gensim.models.Word2Vec(sentences, size=dimensionality, iter=iterations)
    model = gensim.models.Word2Vec(sentences, size=dimensionality, iter=iterations, min_count=3)
    return model

def TrainP2VModel(train_corpus, dimensionality, iterations):
    corpus_tagged = []
    for i in range(0,len(train_corpus)):
        corpus_tagged.append(gensim.models.doc2vec.TaggedDocument(train_corpus[i], [i]))

    model = gensim.models.doc2vec.Doc2Vec(dm=1, size=dimensionality, min_count=2, iter=iterations)
    model.build_vocab(corpus_tagged)
    model.train(corpus_tagged)
    return model

### Assembling vectors for corpus documents

In [63]:
def GetVectors(model, dictionary, dimensionality, corpus):
    dim_sum = 0
    vectors = []
    for i in range(0, len(corpus)):        
        prevector = model[dictionary.doc2bow(corpus[i])]
        dim_sum += len(prevector)
        vector = [0]*dimensionality
        for k,v in prevector:
            vector[k]=v
        vectors.append(vector)
    return vectors

def GetAverageW2Vector(model, doc, dimensionality):
    wordcount = 0
    docvector = [0]*dimensionality
    for word in doc:
        if word in model.vocab:
            wordcount+=1
            docvector=[x + y for x, y in zip(docvector, model[word])]
    docvector=[x / wordcount for x in docvector]
    return docvector

def GetAverageW2VectorsCorpus(model, corpus):
    vectors = []
    for doc in corpus:
        vectors.append(GetAverageW2Vector(model, doc))
    return vectors

def GetWeightedAverageW2Vector(model, weights, dictionary, doc, dimensionality):
    wordcount = 0
    docvector = [0]*dimensionality
    
    d = dict(weights[dictionary.doc2bow(doc)])

    for word in doc:
        if 1==1:
            weight=dictionary.doc2bow([word])
            #if (word in model.vocab) and (len(weight)>0):
            if (word in model.wv.vocab) and (len(weight)>0):
                wordcount+=1
                w=weight[0][0]
                if not (w in d):
                    w=0
                else:
                    w=d[w]
                docvector=[(x + (y*w)) for x, y in zip(docvector, model[word])]
    if wordcount == 0:
        docvector=[x / wordcount for x in docvector]
    return docvector

def GetWeightedAverageW2VectorsCorpus(model, weights, dictionary, corpus, dimensionality):
    vectors = []
    for doc in corpus:
        vectors.append(GetWeightedAverageW2Vector(model, weights, dictionary, doc, dimensionality))
    return vectors

def GetP2VectorsCorpus(model, corpus):
    vectors = []
    for doc in corpus:
        vectors.append(model.infer_vector(doc))
    return vectors

### Obtaining simple search query results

In [5]:
def most_similar_docs(docs, model, doc, topn=10):
    similar_docs = []
    
    inferred_vector = model.infer_vector(doc)
    vecs=model.docvecs.most_similar([inferred_vector], topn=topn)
    
    for i,sim in vecs:
        similar_docs.append(tuple((docs[i], sim)))
    return similar_docs

def most_similar_docs_w2v(docs, model, tfidfmodel, dictionary, vectors, doc, topn=10):
    similar_docs = []

    inferred_vector = GetWeightedAverageW2VectorsCorpus(model, tfidfmodel, dictionary, [doc], dimensionality)
    
    sims=[]
    
    for i in range(0,len(vectors)):
        sims.append((corpus_names[i], cosine_similarity(np.reshape(vectors[i], (1,-1)), np.reshape(inferred_vector, (1,-1)))[0][0]))
                    
    similar_docs=sorted(sims, key=lambda x: x[1], reverse=True)

    return similar_docs[:topn]

def most_similar_docs_lda_lsi_tfidf(docs, model, dictionary, topics, doc, topn=10):
    similar_docs = []

    vectors = GetVectors(model, dictionary, topics, corpus)    
    inferred_vector = GetVectors(model, dictionary, topics, [doc])[0]
    
    sims=[]
    
    for i in range(0,len(vectors)):
        sims.append((corpus_names[i], cosine_similarity(np.reshape(vectors[i], (1,-1)), np.reshape(inferred_vector, (1,-1)))[0][0]))
                    
    similar_docs=sorted(sims, key=lambda x: x[1], reverse=True)
    return similar_docs[:topn]

### Packing results

In [18]:
def pack_similar_documents(docs):
    results = {}
    for key in docs.keys():
        results[key]=docs[key]
    return results

def pack_sims(model_name, id, query_names, query_data, topn=10):
    results={}
    results['Filename']=query_names[id]
    search_results={}
    #search_results['p2v']=most_similar_docs(corpus_names, p2vmodel, query_data, topn=topn)
    #search_results['lsi']=most_similar_docs_lda_lsi_tfidf(corpus_names, lsimodel, lsidictionary, lsi_topics, query_data, topn=topn)
    #search_results['lda']=most_similar_docs_lda_lsi_tfidf(corpus_names, ldamodel, ldadictionary, lda_topics, query_data, topn=topn)
    #search_results['tfidf']=most_similar_docs_lda_lsi_tfidf(corpus_names, tfidfmodel, tfidfdictionary, len(tfidfdictionary), query_data, topn=topn)
    search_results[model_name]=most_similar_docs_w2v(corpus_names, waw2vmodel, w2vtfidfmodel, w2vdictionary, w2vvectors, query_corpus[id], topn=topn)

    results['Results']=pack_similar_documents(search_results)
    
    return results

def pack_it_all(model_name, filename, query_names, query_data, topn=10):
    sims=[]
    for i in range(0,len(query_names)):
        sims.append(pack_sims(model_name, i, query_names, query_data, topn))
        print ('Packed {} - {}'.format(i, query_names[i]))

    res=json.dumps(sims, ensure_ascii=False)

    with open(filename+'.json', 'w', encoding="utf-8") as f:
        print(res, file=f)

In [86]:
def GetVectors_test(model, dictionary, dimensionality, corpus):
    #print(len(corpus))
    #if(len(corpus) is 1): 
    #    print(corpus)
    dim_sum = 0
    vectors = []
    for i in range(0, len(corpus)): 
        #if False: 
        #    temp = list(map(str, corpus[i]))
        #    prevector = model[dictionary.doc2bow(temp)]
        #else:
        prevector = model[dictionary.doc2bow(corpus[i])]
        dim_sum += len(prevector)
        vector = [0]*dimensionality
        for k,v in prevector:
            vector[k]=v
        vectors.append(vector)
    return vectors

def most_similar_docs_lda_lsi_tfidf_test(docs, model, dictionary, topics, doc, topn=10):
    similar_docs = []

    vectors = GetVectors_test(model, dictionary, topics, corpus)
    inferred_vector = GetVectors_test(model, dictionary, topics, [doc])[0]
    #print(inferred_vector)
    
    sims=[]
    
    for i in range(0,len(vectors)):
        sims.append((i, cosine_similarity(np.reshape(vectors[i], (1,-1)), np.reshape(inferred_vector, (1,-1)))[0][0]))
                    
    similar_docs=sorted(sims, key=lambda x: x[1], reverse=True)
    return similar_docs[:10]

def most_similar_docs_w2v_test(docs, model, tfidfmodel, dictionary, vectors, doc, topn=10, temp_name_is = True, temp_name_vectors = None):
    similar_docs = []

    inferred_vector = GetWeightedAverageW2VectorsCorpus(model, tfidfmodel, dictionary, [doc], dimensionality)
    
    sims=[]
    
    for i in range(0,len(vectors)):
        if temp_name_is:
            temp_name = i
        else:
            temp_name = corpus_names[temp_name_vectors[i][0]]
        sims.append((temp_name, cosine_similarity(np.reshape(vectors[i], (1,-1)), np.reshape(inferred_vector, (1,-1)))[0][0]))
                    
    similar_docs=sorted(sims, key=lambda x: x[1], reverse=True)

    # ATTENTION !!!!!!!!!!!!!
    #return similar_docs[:topn]
    return similar_docs[:topn]


def pack_sim_test(model_name, id, query_names, query_data, topn=10):
    results={}
    results['Filename']=query_names[id]
    search_results={}
    
    # TFIDF
    temp_vectors = most_similar_docs_lda_lsi_tfidf_test(corpus_names, tfidfmodel, tfidfdictionary, len(tfidfdictionary), query_data[id], topn=topn)
    # W2V
    #temp_vectors = most_similar_docs_w2v_test(corpus_names, waw2vmodel, w2vtfidfmodel, w2vdictionary, w2vvectors, query_corpus[id], topn=topn)
    temp_temp_corpus = [w2vvectors_out[x[0]] for x in temp_vectors]
    print(len(temp_vectors))
    out_vectors = most_similar_docs_w2v_test(corpus_names, waw2vmodel, w2vtfidfmodel, w2vdictionary, temp_temp_corpus, query_corpus[id], topn=topn, temp_name_is=False, temp_name_vectors = temp_vectors)    
    #print(out_vectors)
    search_results[model_name]=out_vectors
    results['Results']=pack_similar_documents(search_results)
    
    return results

def pack_it_all_test(model_name, filename, query_names, query_data, topn=10):
    sims=[]
    for i in range(0,len(query_names)):
        sims.append(pack_sim_test(model_name, i, query_names, query_data, topn))
        print ('Packed {} - {}'.format(i, query_names[i]))

    res=json.dumps(sims, ensure_ascii=False)
    #print(res)
    with open(filename+'.json', 'w', encoding="utf-8") as f:
        print(res, file=f)

### Run

In [7]:
data_dir = "D:\\Разработка\\Python\\Julius\\data\\"

#mooc_preserved = 'corpus_mooc.txt'
mooc_corpus = 'courses.cor'
mooc_names = 'courses.cfn'

#rpd_preserved = 'test_rpds.txt'
rpd_names='docs.cfn'
rpd_corpus='docs.cor'

### Lemmatization

In [10]:
#morph = pymorphy2.MorphAnalyzer()
#%time build_lemmatized_corpora(data_dir, mooc_preserved, morph)
#%time build_lemmatized_corpora(data_dir, rpd_preserved, morph)

### Read corpus

In [8]:
corpus = list(read_corpus(data_dir + mooc_corpus, tokens_only=True))
corpus_names = list(read_corpus(data_dir + mooc_names, preserve=True))

query_corpus = list(read_corpus(data_dir + rpd_corpus, tokens_only=True))
query_names = list(read_corpus(data_dir + rpd_names, preserve=True))

print('Length corpus: ', len(corpus))
print('Length query_corpus', len(query_corpus))

Length corpus:  1276
Length query_corpus 29


### Train model

In [9]:
print('Training time TF-IDF model')
%time tfidfmodel, tfidfdictionary = TrainTFIDFModel(corpus)

dimensionality = 300
iterations = 55
print('Training time Word2Vec model')
%time waw2vmodel = TrainW2VModel(corpus, dimensionality, iterations)

#%time p2vmodel = TrainP2VModel(corpus, 50, 55)

lda_topics = 25
#%time ldamodel, ldadictionary = TrainLDAModel(corpus)

lsi_topics = 25
#%time lsimodel, lsidictionary = TrainLSIModel(corpus)

Training time TF-IDF model
Wall time: 1.41 s
Training time Word2Vec model
Wall time: 1min 16s


In [11]:
# remove \r from corpus_name
corpus_names = [s.replace('\r', '') for s in corpus_names]

print('length vocabulary w2v:', len(waw2vmodel.wv.vocab))
print('length vocabulary tf-idf:', len(tfidfdictionary))

length vocabulary w2v: 13180
length vocabulary tf-idf: 28811


### Create out vectors of w2v model

In [12]:
outv = gensim.models.KeyedVectors()
outv.vocab = waw2vmodel.wv.vocab  # same
outv.index2word = waw2vmodel.wv.index2word  # same
outv.syn0 = waw2vmodel.syn1neg  # different

### "IN - OUT" matrix example

In [13]:
test_word = 'язык'
print('Vocab:', test_word)

print('IN - IN similar')
display(waw2vmodel.most_similar(positive=[waw2vmodel[test_word]]))
print('OUT - OUT similar')
display(outv.most_similar(positive=[outv[test_word]]))
print('IN - OUT similar')
display(waw2vmodel.most_similar(positive=[outv[test_word]]))
print('OUT - IN similar')
display(outv.most_similar(positive=[waw2vmodel[test_word]]))

Vocab: язык
IN - IN similar


[('язык', 1.0000001192092896),
 ('синтаксис', 0.4058026075363159),
 ('интерпретатор', 0.36988210678100586),
 ('грамматик', 0.3655482828617096),
 ('турбо', 0.3282540440559387),
 ('лексика', 0.32689809799194336),
 ('java', 0.31306377053260803),
 ('лисп', 0.3107827603816986),
 ('космизм', 0.3103848397731781),
 ('конструкция', 0.30966126918792725)]

OUT - OUT similar


[('язык', 1.0),
 ('космизм', 0.7770746350288391),
 ('неродной', 0.7676228880882263),
 ('прагматизм', 0.7639051675796509),
 ('анархизм', 0.7638077139854431),
 ('бердяев', 0.7635451555252075),
 ('зарубежье', 0.7635377645492554),
 ('геополитический', 0.7635335922241211),
 ('нбф', 0.7623878717422485),
 ('omg', 0.7623332142829895)]

IN - OUT similar


[('лп', 0.35911887884140015),
 ('фортран', 0.3075765371322632),
 ('макрообработка', 0.2972302734851837),
 ('sh', 0.2934834361076355),
 ('питон', 0.2705475687980652),
 ('компоновщик', 0.2597852349281311),
 ('sml', 0.2545531392097473),
 ('документный', 0.2526775598526001),
 ('dcl', 0.23822461068630219),
 ('ocl', 0.2283439189195633)]

OUT - IN similar


[('русский', 0.13989201188087463),
 ('си', 0.13868120312690735),
 ('неавтоматный', 0.138381689786911),
 ('английский', 0.13732579350471497),
 ('pascal', 0.10128071904182434),
 ('документный', 0.0976657122373581),
 ('babel', 0.09709149599075317),
 ('пролог', 0.09007721394300461),
 ('семиотика', 0.08583243936300278),
 ('python', 0.08485482633113861)]

In [42]:
# Обучение на in матрице
%time w2vvectors = GetWeightedAverageW2VectorsCorpus(waw2vmodel, tfidfmodel, tfidfdictionary, corpus, dimensionality)

# Обучение на out матрице
#%time w2vvectors = GetWeightedAverageW2VectorsCorpus(outv, tfidfmodel, tfidfdictionary, corpus, dimensionality)

# Temp 
%time w2vvectors_out = GetWeightedAverageW2VectorsCorpus(outv, tfidfmodel, tfidfdictionary, corpus, dimensionality)

Wall time: 15min 53s
Wall time: 15min 41s


### Export model 

In [87]:
w2vtfidfmodel = tfidfmodel
w2vdictionary = tfidfdictionary

%time pack_it_all_test(model_name='w2v_tfidf_xattab_in_in_outrange_top10_300_i55_cm3', filename='w2v_tfidf_xattab_in_in_outrange_top10_300_i55_cm3', query_names=query_names, query_data=query_corpus, topn=20)

10
Packed 0 - 21_РПД_Теория вероятностей и математическая статистика
10
Packed 1 - 15_РПД_Защита информации
10
Packed 2 - 24_ РПД _Алгоритмы и анализ сложности 
10
Packed 3 - 4_РПД Экономика
10
Packed 4 - 30_ РПД _Объектно-ориентированный анализ и программирование
10
Packed 5 - РПД БИ Маг Переговоры
10
Packed 6 - РПД _Информационные системы и технологии ПИ_09.03.03(2)
10
Packed 7 - 6_РПД_Линейная алгебра и аналитическая геометрия
10
Packed 8 - 10_РПД _Базы данных
10
Packed 9 - 9_РПД_Операционные системы
10
Packed 10 - 41_1_РПД _Теория автоматов и формальных языков
10
Packed 11 - 11_РПД _Программирование
10
Packed 12 - 37_РПД _Программная инженерия
10
Packed 13 - 15_РПД_Сети и телекоммуникации
10
Packed 14 - 31_РПД _Программирование на Java
10
Packed 15 - РПД Менеджмент 38.03.05 очка
10
Packed 16 - РПД_ теория принятия решений_ маг БИ (вер2)
10
Packed 17 - 5_РПД _Математический анализ, Дифференциальные и разностные уравнения
10
Packed 18 - 35_РПД _ Локальные и глобальные вычислительные 